### Environmental Variables

In [5]:
# A lowest level of capability for including the P-S cell relation, skip otherwise:
CAPABILITY_LIMIT = 0.1

# Set max number of BB partners per direction:
MAX_BB_PARTNERS = 6

# Set max number of external cells per partner and per direction:
MAX_EXTERNAL_CELLS_SECONDARY_GNB = 10

### Input from Flow Automation

In [6]:
selectivity = 'full' # full or partial - decision from the customer
node_list = [] # List of nodes to exclude from the total
unwanted_bb_links = [] #List of links not to create
mandatory_bb_links = [] # List of links that must be created

def set_node_selection(selection, primary_nodes, unwanted_links, mandatory_links):
    selectivity = selection
    node_list = primary_nodes
    unwanted_bb_links = unwanted_links
    mandatory_bb_links = mandatory_links

### Load topology data

In [7]:
def fetch_topology_data(filename):
    return open_file(filename)

def get_TMO_topology_data(source_dir):
    df = fetch_topology_data(source_dir + '/cucp_cellcu_merge.csv')
    df['cell_fdn'] = df['meFdn'] + ',' + df['refId']
    df.rename({'CellName': 'nRCellCUId', 'meFdn': 'node_fdn',
                       'GNBID': 'gNBId', 'GNBID_LENGTH': 'gNBIdLength'}, axis=1, inplace=True)
    df.drop(['parentRefId', 'refId', 'cellLocalId', 'MCC', 'MNC'], axis=1, inplace=True)
    
    return df

def get_SwissCom_topology_data(source_dir):
    df_NRCellCU = fetch_topology_data(source_dir + '/NRCellCU.csv')
    df_NRCellCU['NodeName'] = df_NRCellCU['fdn'].apply(lambda x: split_name(x, 'ManagedElement='))
    df_NRCellCU.drop(['nRFrequencyRef'], axis=1, inplace=True)
    df_NRCellCU.rename({'fdn': 'cell_fdn'}, axis=1, inplace=True)
    
    df_GNBCUCPFunction = fetch_topology_data(source_dir + '/GNBCUCPFunction.csv')
    df_GNBCUCPFunction['NodeName'] = df_GNBCUCPFunction['fdn'].apply(lambda x: split_name(x, 'ManagedElement='))
    df_GNBCUCPFunction.rename({'fdn': 'node_fdn'}, axis=1, inplace=True)
    
    df = df_NRCellCU.merge(df_GNBCUCPFunction, left_on='NodeName', right_on='NodeName')    
    return df

### Trigger MO Action for nodes of interest

In [8]:
def get_nodes_to_evaluate(selectivity, node_list, df_topology):
    df = pd.DataFrame()
    if selectivity == 'full':
        df = df_topology[ ~df_topology['NodeName'].isin(node_list) ]
    elif selectivity == 'partial':
        df = df_topology[ df_topology['NodeName'].isin(node_list) ]
    return df

def trigger_mo_action(df_evaluation):
    count = 0    
    for index, row in df_evaluation.iterrows():
        # The MO action would be triggered through NCMP per cell
        # The node and cell details would also be pushed to the PM Handler to filter incoming data so we only consume what we are interested in. 
        # print ('MO action triggered for cell', row['nRCellCUId'], 'on node', row['NodeName'])
        count+=1
    
    return 1, count

### Load Coverage data

In [9]:
def fetch_coverage_data(source_dir, date):
    return open_file(source_dir + '/coverage_data_' + date +'.csv')

def decode_hitrate(arcBalance):
     return arcBalance
# def decoded_usefulness(coded_usefulness):
#     # Sets the constant use (basically only a scaling):
#     k = 0.2
#     return (coded_usefulness / k) ** (1 / (k - 1))

def convert_hitrate(df):
    df['hitrate'] = [
        decode_hitrate(val)
        for val in df['arcBalance']
    ]
    return df

def get_coverage_data(source_dir, date, df_top_enrich):
    # Fetch the data from the PM Handler (csv in this case)
    df = fetch_coverage_data(source_dir, date)
    
    # Enrich the coverage data with the node and cell info from the topology
    df = df_top_enrich.merge(df, right_on=['src_nCI'], left_on=['nCI'])
    df.rename({'gNBId': 'src_gNBId'}, axis=1, inplace=True)
    df.drop(['src_nRFrequencyRef', 'tgt_nRFrequencyRef', 'nCI'], axis=1, inplace=True)
    
     # Filter out rows where the source and target nodes are the same. 
    df = (
        df[ df["src_gNBId"] != df["tgt_gNBId"] ]
    ).copy()
    
    # Decode the hitrate
    if not df.empty:
        df = convert_hitrate(df)
    
    df.reset_index(drop=True, inplace=True)
    return df

### Load Capacity data

In [10]:
def fetch_capacity_data(df_required_ids, source_dir, date):
    df = open_file(source_dir + '/capacity_data_' + date + '.csv')
    # Pull the node name and cell name 
    df['NodeName'] = df['fdn'].apply(lambda x: split_name(x, 'ManagedElement='))
    df['CellName'] = df['fdn'].apply(lambda x: split_name(x, 'NRCellDU='))
    
    # Filter for the data needed
    df = df.merge(df_required_ids, left_on=['NodeName','CellName'], right_on=['NodeName', 'nRCellCUId'])
    
    # Drop the columns we dont need
    df.drop(['fdn', 'timestamp', 'NodeName', 'nRCellCUId', 'CellName'], axis=1, inplace=True)

    return df

def calculate_kpis(df_pm):
    df_pm["RBSymFree"] = (
        df_pm["pmMacRBSymAvailDl"] - df_pm["pmMacRBSymUsedPdschTypeA"] )
    return df_pm


def get_model(file_name):
    df = pd.read_csv(file_name, sep=',')
    return df.loc[0]["intercept"], df.loc[0]["RBSymFree"], df.loc[1]["RBSymFree"]

def normalize_data(df, normalize_columns, normalize_constants):
    for index, col in enumerate(normalize_columns):
        df[col + "Norm"] = [
            (value / normalize_constants[index]) for value in df[col]
        ]
    return df

def add_predicted_cell_capacity(model_file_name, df_capacity_data):
    intercept, coefficient_0, normalize_constant_0 = get_model( model_file_name )
    df_capacity_pred = normalize_data(
        df_capacity_data, ["RBSymFree"], [normalize_constant_0]
    )

    df_capacity_pred["predictedCapacity"] = (
        intercept + coefficient_0 * df_capacity_pred["RBSymFreeNorm"]
    )
    return df_capacity_pred


def get_capacity_data(df_required_ids, df_top_enrich, source_dir, date):
    df = fetch_capacity_data(df_required_ids, source_dir, date)
    df = calculate_kpis(df)
    df = add_predicted_cell_capacity('modelParametersLinRegr_60.csv', df)
    return df

### Build Usability Matrix

In [ ]:
def build_usability_matrix(coverage_data, capacity_data, capability_limit):
    usability_matrix = coverage_data.copy()
    # Get predicted capacity for matching sNCI
    usability_matrix = capacity_data.merge(coverage_data, how="left", on="tgt_nCI").drop_duplicates()
    # Change NaN to 0, if any
    usability_matrix["predictedCapacity"].fillna(0, inplace=True)
    # Multiply hit rate with predicted capacity
    usability_matrix["usability"] = (
        usability_matrix["hitrate"] * usability_matrix["predictedCapacity"]
    )
    
    usability_matrix["usability"].where(
        usability_matrix["usability"] > capability_limit, other=0, inplace=True
    )
    
     # Clean up the resulting data frame
    usability_matrix.rename({'src_gNBId': 'primary_gnb', 
                         'tgt_gNBId_x': 'secondary_gnb',
                        'src_nCI': 'pNCI',
                        'tgt_nCI': 'sNCI'}, axis=1, inplace=True)

    usability_matrix.drop(
        columns=[
            "src_nRCellCUId",
            "tgt_nRCellCUId",
            #"tgt_pLMNId",
            "tgt_gNBIdLength",
            "arcBalance",
            #"hitrate",
            "tgt_gNBId_y",
            "pmMacRBSymUsedPdschTypeA",
            "pmMacRBSymAvailDl",
            "RBSymFree",
            "RBSymFreeNorm",
            "predictedCapacity",
        ],
        inplace=True,
    )    
    return usability_matrix

### Create link value list

In [ ]:
def create_link_value_list(usability_matrix):
    # Remove zero usability rows:
    bb_link_value_list = usability_matrix[usability_matrix["usability"] > 0].copy()
    # Add tuple with (primary_gnb, secondary_gnb):
    bb_link_value_list["gNbs"] = list(
        zip(bb_link_value_list["primary_gnb"], bb_link_value_list["secondary_gnb"])
    )
    # Sort the entire data frame based on the columns of gNBs and the usability
    bb_link_value_list.sort_values(
        ["primary_gnb", "secondary_gnb", "usability"], inplace=True, ascending=False
    )
    # Aggregate usability value per eNB pair and sort the list in descending usability order
    bb_link_value_list = (
        bb_link_value_list[["primary_gnb", "secondary_gnb", "gNbs", "usability", "hitrate"]]
        .groupby(["primary_gnb", "secondary_gnb", "gNbs"])
        .agg({"usability": "sum", "hitrate": "mean"})
        .sort_values("usability", ascending=False)
    )
    # Set numerical index
    bb_link_value_list.reset_index(inplace=True)
    # Add column indicating if link is used in configuration
    bb_link_value_list["selected"] = False
    
    return bb_link_value_list

### Mark Unwanted links

In [ ]:
def mark_unwanted_bb_link(df_link_value, unwanted_bb_links, df_topology):
    df_link_value['unwanted'] = False
    unwanted_links_gnbids = []
    for unwanted_bb_link in unwanted_bb_links:
        bb_0_id = df_topology[df_topology['NodeName'] == unwanted_bb_link[0]]['gNBId'].values[0]
        bb_1_id = df_topology[df_topology['NodeName'] == unwanted_bb_link[1]]['gNBId'].values[0]
        unwanted_links_gnbids.append((bb_0_id, bb_1_id))
    
    
    df_link_value.loc[ df_link_value.gNbs.isin(unwanted_links_gnbids), 'unwanted'] = True

    df_link_value.reset_index(drop=True, inplace=True)
    return df_link_value

### Mark Mandatory links

In [ ]:
def mark_mandatory_bb_links(bb_link_value_list, mandatory_bb_links, df_topology):
    bb_link_value_list['mandatory'] = False
    for bb in mandatory_bb_links:
        bb_0_id = df_topology[df_topology['NodeName'] == bb[0]]['gNBId'].values[0]
        bb_1_id = df_topology[df_topology['NodeName'] == bb[1]]['gNBId'].values[0]
        bb = (bb_0_id, bb_1_id)
#         unwanted = bb_link_value_list[bb_link_value_list["gNbs"] == bb]['Unwanted']
#         print(unwanted)
        if bb in set(bb_link_value_list["gNbs"]):
            bb_link_value_list.loc[bb_link_value_list["gNbs"] == bb, "selected"] = True
            bb_link_value_list.loc[bb_link_value_list["gNbs"] == bb, "mandatory"] = True
        else:
            bb_link_value_list.loc[len(bb_link_value_list.index)] = [
                bb,
                0,
                bb[0],
                bb[1],
                True,
                False,
                True
            ]
    sorted_bb_link = bb_link_value_list.sort_values(
        ["selected", "usability"], ascending=False
    )
    sorted_bb_link.reset_index(drop=True, inplace=True)
    return sorted_bb_link

### Select Best links

In [ ]:
def get_unique_gnb_list(bb_link_value_list):
    if bb_link_value_list.empty:
        unique_gnbs = []
    else:
        unique_gnbs = sorted(
            list(set(bb_link_value_list["gNbs"].apply(lambda x: [x[0], x[1]]).sum()))
        )
    return unique_gnbs

def assign_bb_links_inner(gnbs, gnb0, gnb1, unwanted, max_bb_partners):
    # Create arrays initialized with zero links per gNB
    links_per_gnb_to_sec_col = np.zeros(len(gnbs))
    links_per_gnb_to_prim_col = np.zeros(len(gnbs))
    # Create array initialized to False for every link usage
    link_used_col = np.full(len(gnb0), False)
    # Go through the gNB pairs (= link) and see if it is assignable
    for link in range(len(link_used_col)):
        # Check if gNb0 and gNb1 both has fewer links than allowed limit
        index_0 = np.where(gnbs == gnb0[link])[0]
        index_1 = np.where(gnbs == gnb1[link])[0]
        unwanted_state = unwanted[link]
        if (links_per_gnb_to_sec_col[index_0][0] < max_bb_partners) and (
            links_per_gnb_to_prim_col[index_1][0] < max_bb_partners):
            
            if unwanted_state == False:
                # Both gNBs have available links, set link to be used and increment number of links
                link_used_col[link] = True
                links_per_gnb_to_sec_col[index_0] += 1
                links_per_gnb_to_prim_col[index_1] += 1
    return link_used_col


def assign_bb_links(bb_link_value_list, max_bb_partners):
    unique_gnbs = get_unique_gnb_list(bb_link_value_list)
    bb_link_value_list.loc[:, "selected"] = assign_bb_links_inner(
        np.array(unique_gnbs),
        bb_link_value_list["primary_gnb"].values,
        bb_link_value_list["secondary_gnb"].values,
        bb_link_value_list["unwanted"].values,
        max_bb_partners,
    ).tolist()
    return bb_link_value_list

### Enrich selected links with node names

In [ ]:
def enrich_link_selection(bb_link_resulted_df, df_topology):
    df_node_names_mapping = df_topology[['NodeName', 'gNBId']].drop_duplicates()

    df_links_node_names = bb_link_resulted_df.merge(df_node_names_mapping, how="inner",
                                    right_on=['gNBId'],
                                    left_on=['primary_gnb'])

    df_links_node_names = df_links_node_names.merge(df_node_names_mapping, how="inner",
                                    right_on=['gNBId'],
                                    left_on=['secondary_gnb'])

    df_links_node_names.rename({'NodeName_x': 'primary_node_name', 
                            'NodeName_y': 'secondary_node_name'}, axis=1, inplace=True)

    df_links_node_names.drop(columns=["gNBId_x", "gNBId_y"], inplace=True)

    return df_links_node_names


### "Main" method calling pulling the algorithm together.

In [11]:
def run_algorithm(source_dir, date):
    # Load the topology data
    df_topology = get_TMO_topology_data(source_dir)
    #df_topology = get_SwissCom_topology_data(source_dir)


    # Trigger the MO actions for the nodes we are interested in
    df_evaluation = get_nodes_to_evaluate(selectivity, node_list, df_topology)
    state, count = trigger_mo_action(df_evaluation)
    if state:
        print(str(count) + ' MO Actions triggered')
    else:
        print('MO Action trigger failed')


    # Load the coverage data
    df_coverage_data = get_coverage_data(source_dir, date, df_evaluation[['nCI', 'gNBId']])


    # Load Capacity data for the secondary nodes only. 
    # We only need data for the target cells so we can filter based on the coverage data
    df_required_ids = df_coverage_data[['tgt_nCI', 'tgt_gNBId']].drop_duplicates()
    # Enrich the required data using topology so we can use it to filter the incoming PM data
    df_top_enrich = df_topology[['nCI', 'nRCellCUId', 'NodeName', 'gNBId']]
    df_required_ids = df_required_ids.merge(df_top_enrich,
                                    left_on=['tgt_gNBId','tgt_nCI'],
                                    right_on=['gNBId', 'nCI'])
    df_required_ids.drop(['gNBId', 'nCI'], axis=1, inplace=True)
    df_capacity_data = get_capacity_data(df_required_ids, df_top_enrich, source_dir, date)
    

    # Use the data to build the usabililty matrix
    df_usability_matrix = build_usability_matrix(df_coverage_data, df_capacity_data, CAPABILITY_LIMIT)


    # Create link value list
    df_link_value = create_link_value_list(df_usability_matrix)


    # Mark unwanted links
    df_link_value = mark_unwanted_bb_link( df_link_value, unwanted_bb_links, df_topology )


    # Mark Mandatory links
    df_link_value = mark_mandatory_bb_links( df_link_value, mandatory_bb_links, df_topology )
    
    
    # Select the priority links
    bb_link_resulted_df = assign_bb_links(df_link_value, MAX_BB_PARTNERS )


    # Enrich the selected links with node names
    df_links_node_names = enrich_link_selection(bb_link_resulted_df, df_topology)
    
    return df_links_node_names